In [ ]:
# install requirements from requirements_fingerprinting.txt
%pip install -r requirements_fingerprinting.txt

In [1]:
import data_sanitize
import importlib
importlib.reload(data_sanitize)
import pandas as pd

training_data = data_sanitize.get_saved_training_data()
#traces = data_sanitize.get_traces()
#training_data = data_sanitize.get_training_data_from_traces(traces)

Reading saved data: 100%|██████████| 100/100 [00:00<00:00, 4110.33it/s]


In [22]:
# convert the training data which is a numpy array
            # self.label,
            # self.num_outgoing,
            # self.num_incoming,
            # self.total_packets,
            # self.outgoing_ratio,
            # self.incoming_ratio,
            # self.outgoing_bytes,
            # self.incoming_bytes,
            # self.avg_outgoing_freq,
            # self.avg_incoming_freq,
            # self.std_outgoing_freq,
            # self.std_incoming_freq,
            # self.avg_outgoing_bytes,
            # self.avg_incoming_bytes,
            # self.std_outgoing_bytes,
            # self.std_incoming_bytes,
            # self.min_outgoing_bytes,
            # self.min_incoming_bytes,
            # self.max_outgoing_bytes,
            # self.max_incoming_bytes,
            # self.min_outgoing_freq,
            # self.min_incoming_freq,
            # self.max_outgoing_freq,
            # self.max_incoming_freq,
            # self.avg_outgoing_order,
            # self.std_outgoing_order,
df = pd.DataFrame(training_data, columns=[
    'grid',
    'num_outgoing',
    'num_incoming',
    'total_packets',
    'outgoing_ratio',
    'incoming_ratio',
    'outgoing_bytes',
    'incoming_bytes',
    'avg_outgoing_freq',
    'avg_incoming_freq',
    'std_outgoing_freq',
    'std_incoming_freq',
    'avg_outgoing_bytes',
    'avg_incoming_bytes',
    'std_outgoing_bytes',
    'std_incoming_bytes',
    'min_outgoing_bytes',
    'min_incoming_bytes',
    'max_outgoing_bytes',
    'max_incoming_bytes',
    'min_outgoing_freq',
    'min_incoming_freq',
    'max_outgoing_freq',
    'max_incoming_freq',
    'avg_outgoing_order',
    'std_outgoing_order',
])
df.head()

,grid,num_outgoing,num_incoming,total_packets,outgoing_ratio,incoming_ratio,outgoing_bytes,incoming_bytes,avg_outgoing_freq,avg_incoming_freq,...,min_outgoing_bytes,min_incoming_bytes,max_outgoing_bytes,max_incoming_bytes,min_outgoing_freq,min_incoming_freq,max_outgoing_freq,max_incoming_freq,avg_outgoing_order,std_outgoing_order
0,56.0,157.0,133.0,292.0,0.537671,0.455479,414308.0,40486.0,0.042820,0.050942,...,52.0,52.0,14532.0,2948.0,0.000024,0.000023,0.277571,0.320129,147.229299,84.360454
1,56.0,183.0,169.0,359.0,0.509749,0.470752,417216.0,43430.0,0.183483,0.199041,...,52.0,52.0,14532.0,2948.0,0.000019,0.000030,3.837083,3.837594,178.732240,102.639799
2,56.0,141.0,121.0,264.0,0.534091,0.458333,404004.0,37718.0,0.042571,0.049325,...,52.0,52.0,13084.0,2948.0,0.000007,0.000041,0.349570,0.349689,132.886525,75.227701
3,56.0,193.0,163.0,359.0,0.537604,0.454039,416870.0,39366.0,0.079566,0.094301,...,52.0,52.0,11362.0,2948.0,0.000006,0.000025,0.819853,0.863817,181.772021,103.706890
4,56.0,152.0,131.0,285.0,0.533333,0.459649,404774.0,38238.0,0.038934,0.045240,...,52.0,52.0,13084.0,2948.0,0.000022,0.000019,0.283542,0.283656,142.250000,81.829351


In [17]:
# what are the unique grids in the dataset?
label_count = len(df['grid'].unique())
label_count

100

In [65]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Before feature selection: ", X_train.shape)
selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

# fit a random forest classifier to the training data
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# evaluate the model on the testing data
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

Before feature selection:  (4000, 25)
After feature selection:  (4000, 8)


0.737

In [42]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Before feature selection: ", X_train.shape)
selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])

# create the parameter grid
param_grid = {
    'clf__n_estimators': [100, 200, 300, 400, 500],
    'clf__max_depth': [None, 5, 10, 15, 20, 25, 30],
    'clf__min_samples_split': [2, 5, 10, 15, 20],
    'clf__min_samples_leaf': [1, 2, 5, 10, 15, 20],
    'clf__max_features': ['sqrt', 'log2', None]
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

Before feature selection:  (4000, 25)
After feature selection:  (4000, 8)
Fitting 5 folds for each of 3150 candidates, totalling 15750 fits


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [None, 5, 10, 15, 20, 25, 30],
                         'clf__max_features': ['sqrt', 'log2', None],
                         'clf__min_samples_leaf': [1, 2, 5, 10, 15, 20],
                         'clf__min_samples_split': [2, 5, 10, 15, 20],
                         'clf__n_estimators': [100, 200, 300, 400, 500]},
             verbose=1)

In [43]:
# print the best parameters, with feature selection: selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
print("Best params: ", grid_search.best_params_)

# print the best score
print("Best score: ", grid_search.best_score_)

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

Best params:  {'clf__max_depth': 20, 'clf__max_features': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 400}
Best score:  0.752


0.775

In [44]:
# print the best parameters
print(grid_search.best_params_)

# print the best score
print(grid_search.best_score_)

print(accuracy_score(y_test, y_pred))

{'clf__max_depth': 20, 'clf__max_features': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 400}
0.752
0.775


In [66]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
# Gaussian NB
import tensorflow as tf
# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Before feature selection: ", X_train.shape)
# selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
# X_train = selector.transform(X_train)
# X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(label_count, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.fit(X_train, y_train, epochs=100, verbose=1)


Before feature selection:  (4000, 25)
After feature selection:  (4000, 25)
Epoch 1/100
125/125 [==============================] - 1s 5ms/step - loss: 4.1925 - sparse_categorical_accuracy: 0.0585
Epoch 2/100
125/125 [==============================] - 1s 5ms/step - loss: 3.4422 - sparse_categorical_accuracy: 0.1443
Epoch 3/100
125/125 [==============================] - 1s 5ms/step - loss: 2.9724 - sparse_categorical_accuracy: 0.2377
Epoch 4/100
125/125 [==============================] - 1s 5ms/step - loss: 2.6634 - sparse_categorical_accuracy: 0.2940
Epoch 5/100
125/125 [==============================] - 1s 5ms/step - loss: 2.4471 - sparse_categorical_accuracy: 0.3310
Epoch 6/100
125/125 [==============================] - 1s 4ms/step - loss: 2.2921 - sparse_categorical_accuracy: 0.3640
Epoch 7/100
125/125 [==============================] - 1s 5ms/step - loss: 2.1668 - sparse_categorical_accuracy: 0.3947
Epoch 8/100
125/125 [==============================] - 1s 5ms/step - loss: 2.0677 - s

In [67]:
model.evaluate(X_test, y_test, verbose=1)

32/32 [==============================] - 0s 6ms/step - loss: 1.5716 - sparse_categorical_accuracy: 0.6230


[1.5716345310211182, 0.6230000257492065]

In [69]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
# MLP classifier
from sklearn.neural_network import MLPClassifier
# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Before feature selection: ", X_train.shape)
selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# fit a random forest classifier to the training data
rf = MLPClassifier()
rf.fit(X_train, y_train)

# evaluate the model on the testing data
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)


Before feature selection:  (4000, 25)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


After feature selection:  (4000, 7)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.67

In [ ]:
# we now try MLP classifier
from sklearn.neural_network import MLPClassifier

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MLPClassifier())
])

# create the parameter grid
param_grid = {
    'clf__hidden_layer_sizes': [(100,), (200,), (300,), (400,), (500,)],
    'clf__activation': ['identity', 'logistic', 'tanh', 'relu'],
    'clf__solver': ['lbfgs', 'sgd', 'adam'],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'clf__learning_rate': ['constant', 'invscaling', 'adaptive']
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

# print the best parameters
grid_search.best_params_

# print the best score
grid_search.best_score_

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
# We know try a KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

# create the parameter grid
param_grid = {
    'clf__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'clf__weights': ['uniform', 'distance'],
    'clf__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__leaf_size': [10, 20, 30, 40, 50],
    'clf__p': [1, 2]
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

# print the best parameters
grid_search.best_params_

# print the best score
grid_search.best_score_

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)